# Tensorflow를 사용한 AlexNet 구현

Tensorflow를 사용해서 AlexNet을 구현하려고 한다. 필요한 라이브러리를 미리 import한다. import한 라이브러리는 아래와 같다.

In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

본격적인 모델을 짠다. 모델 레이어에 대한 설명은 최종적으로 모델을 다 만든 후 summary를 돌린 후 진행한다.

우선 class 수를 정해준다. 강의안에 주어진 그림을 바탕으로 짜므로 수는 1000으로 지정한다.

모델의 구체적인 코드는 아래와 같다.

In [2]:
n_classes  = 1000

model = Sequential()

# convolution layer1
model.add(Conv2D(96,(11,11), strides = (4,4), padding = 'valid',activation = 'relu',input_shape = [227,227,3]))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))

# convolution layer2
model.add(Conv2D(256,(5,5), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))

# convolution layer3
model.add(Conv2D(384, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))

# convolution layer4
model.add(Conv2D(384, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))

# convolution layer5
model.add(Conv2D(256, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))

# fully connected layer1
model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))

# fully connected layer2
model.add(Dense(4096,activation = 'relu'))

# fully connected layer3
model.add(Dense(n_classes, activation = 'softmax'))


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

전체 모델구조는 위와 같다. 총 5개의 convolution layer와 3개의 fully connected layer로 구성되어있다. 

우선 첫 번째 convolution layer의 코드를 작성할 때는 꼭 input data의 size를 입력해주어야 한다.96개의 11 * 11 * 3 사이즈 필터를 사용하며 227 * 227 * 3 사이즈의 이미지를 55 * 55 * 96 사이즈로 바꿔준다. 이 때 파라미터의 개수는 34944개로 앞선 과제에서 계산한 것과 같은 값이 나온다.

이 후의 layer부터는 input size를 설정하지 않아도 된다. 

첫 번째 convolution layer를 거친 output은 maxpooling을 거처 사이즈를 줄여준다. maxpooling의 경우, 학습되는 weight가 없으므로 파라미터는 없다. 코드에서 설정한 pool size와 stride로 나온 결과는 27 * 27 * 96 으로 H와 W는 절반으로 줄었으나 C는 줄지 않았다. 

두 번째 convolution layer는 27 * 27 * 96 사이즈의 데이터를 input으로 받는다. 이 때, 데이터는 총 256개의 5 * 5 * 96 사이즈 필터를 거처 27 * 27 * 256 사이즈의 데이터로 바꿔준다. 이 때, 필터를 거쳤음에도 데이터의 H와 W는 변함이 없는데 이는 padding과 stride를 적절히 설정했기 때문이다. 이 때 파라미터는 614656개로 역시 이전 과제와 동일하다.

다시 이 output을 input으로 받아, pool size를  3 * 3으로, stride를 2로 설정한 maxpooling 과정을 거치면 13 * 13 * 256 사이즈의 output이 만들어진다. 위와 동일한 이유로 파라미터의 개수는 0이다.

세 번째 convolution layer는 이 output을 input으로 받으며, 총 384개의 3 * 3 * 256 사이즈 필터를 거처 13 * 13 * 384의 output을 도출한다. 이 때 padding과 stride는 1이다. 이 때 파라미터는 885120개이다.

네 번째 convolution layer는 세 번째 convolution layer의 output을 input으로 받는다. 총 384개의 3 * 3 * 384 사이즈의 필터를 거처 13 * 13 * 384 사이즈의 output을 만들며, 이 때 stride와 padding은 앞과 동일하게 각각 1 이다. 이 때 파라미터는 1327448개이다.

다섯 번째 convolution layer는 이전에 도출한 output을 input으로 받으며 256개의 3 * 3 * 384개의 필터를 갖는다. stride와 padding은 역시 1이며 이 때 파라미터는 884992개이다. 다섯 번째 convolution layer를 통해 13 * 13 * 256 사이즈의 output이 도출된다.

이 output은 다시 한번 pooling size가 3 * 3, stride가 2인 maxpooling 과정을 거처 6 * 6 * 256 사이즈로 바뀌게 된다.

이 후, flatten 과정을 거치면 6 * 6 * 256 사이즈의 데이터는 N * 9216 형태의 데이터로 바뀌게 된다. 그리고 그 다음부터는 우리가 이전에 배웠던 기법인 fully connected layer를 만들어주면 된다.

첫 번째 fully connected layer에서의 input은 N * 9216 이며 output은 9216 * 4096이 된다. 이 때, 총 파라미터는 37752832개가 나온다. 자세한 구조는 6주, 7주에 배웠던 NN을 참고하면 된다.

두 번째 fully connected layer와 세 번째 fully connected layer에서는 각각 N * 4096 사이즈의 input을 받으며 이 때 output은 N * 4096, N * 1000이 된다. 총 파라미터는 16781312개, 4097000개가 된다. 

최종적으로 이 모델의 파라미터는 62,378,344개이다.

그렇다면 최종적으로 구현된 이 모델에 data를 넣어 학습시킨다면 과연 잘 예측할 수 있을까? 정답은 '아니오'일 것이다. 실제로 이 모델을 그대로 가져다 쓰면 accuracy는 현저하게 떨어질 것이다. 그 이유는 overfitting 됐기 때문이다. 이를 막기 위해선 layer 중간중간 dropout을 추가한다든지, batchnormalization 등을 추가하여 과적합되는 것을 막아야 한다. 

실제 AlexNet이 만들어질 때, 역시 이 overfitting 문제를 염려했고, 해결책으로 dropout과 data augmentation을 사용했다. 추가로 data augmentation 시, 주의해야하는 점은 label-preserving transformation을 고려해야한다는 점이다. 즉, 이미지를 늘리는 과정에서 변형한 이미지가 기존 이미지의 label을 그대로 가져야 한다는 것이다. 예를 들어 6을 상하반전하면 9가 된다. 이는 머신이 9라고 잘못 학습할 수도 있게된다. 그래서 기존의 label을 유지하며 학습하는 것을 고려해야 한다.

### https://oi.readthedocs.io/en/latest/computer_vision/cnn/alexnet.html



In [ ]:
n_classes  = 1000

model = Sequential()

# convolution layer1
model.add(Conv2D(96,(11,11), strides = (4,4), padding = 'valid',activation = 'relu',input_shape = [227,227,3]))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
model.add(Dropout(0.5))

# convolution layer2
model.add(Conv2D(256,(5,5), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
model.add(Dropout(0.5))

# convolution layer3
model.add(Conv2D(384, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(Dropout(0.5))

# convolution layer4
model.add(Conv2D(384, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(Dropout(0.5))

# convolution layer5
model.add(Conv2D(256, (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = (2,2)))
model.add(Dropout(0.5))

# fully connected layer1
model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))

# fully connected layer2
model.add(Dense(4096,activation = 'relu'))
model.add(Dropout(0.5))

# fully connected layer3
model.add(Dense(n_classes, activation = 'softmax'))
model.add(Dropout(0.5))

위처럼 중간중간 dropout을 넣어주게 되면 overfitting을 막음으로써 더 성능좋은 모델을 만들 수 있게 된다.